In [1]:
from seisLM.data_pipeline.foreshock_aftershock_dataset import (
  create_foreshock_aftershock_datasets
)
from seisLM.data_pipeline.foreshock_aftershock_dataloaders import (
  prepare_foreshock_aftershock_dataloaders)
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
NUM_CLASSES = 4
BATCH_SIZE = 8
EVENT_SPLIT_METHOD = 'temporal'
COMPONENT_ORDER = 'ZNE'
DIMENSION_ORDER = 'NCW'

In [3]:
loaders = prepare_foreshock_aftershock_dataloaders(
  num_classes=NUM_CLASSES,
  batch_size=BATCH_SIZE,
  event_split_method=EVENT_SPLIT_METHOD,
  component_order=COMPONENT_ORDER,
  standardize=True,
)


Seed set to 42
Seed set to 42


In [4]:
import json
import ml_collections
import torch
from seisLM.model.task_specific.foreshock_aftershock_models import ShockClassifierLit #Wav2Vec2ForSequenceClassification

def get_seisLM_config():

  config_path = '/scicore/home/dokman0000/liu0003/projects/seisLM/seisLM/configs/foreshock_aftershock/seisLM_shock_classifier.json'
  with open(config_path, "r", encoding="utf-8") as f:
    config = json.load(f)
  config = ml_collections.ConfigDict(config)

  config.model_args.num_classes = NUM_CLASSES
  config.model_args.use_weighted_layer_sum = True
  return config

seisLM_config = get_seisLM_config()

models = {}

models['seisLM'] = ShockClassifierLit(
  model_name='Wav2Vec2ForSequenceClassification',
  model_config=seisLM_config.model_args,
  training_config=seisLM_config.trainer_args,
  ).model



In [5]:
def get_conv1d_config():
  config_path = '/scicore/home/dokman0000/liu0003/projects/seisLM/seisLM/configs/foreshock_aftershock/conv1d_shock_classifier.json'
  with open(config_path, "r", encoding="utf-8") as f:
    config = json.load(f)
  config = ml_collections.ConfigDict(config)
  config.model_args.num_classes = NUM_CLASSES
  return config

conv1d_config = get_conv1d_config()

models['cnn'] = ShockClassifierLit(
  model_name='Conv1DShockClassifier',
  model_config=conv1d_config.model_args,
  training_config=conv1d_config.trainer_args,
  ).model

In [10]:
waveforms, labels = next(iter(loaders['train']))


key = 'cnn'
# key = 'seisLM'
model = models[key]


logits = models[key](waveforms)
loss = torch.nn.functional.cross_entropy(logits, labels)
predicted_labels = torch.argmax(logits, 1)

logits

tensor([[ 0.1296, -0.2033,  0.0850,  0.0575],
        [ 0.0347, -0.3280,  0.0400,  0.0650],
        [ 0.0973, -0.2938,  0.0784,  0.0562],
        [-0.0325, -0.4147,  0.0696,  0.0440],
        [ 0.1388, -0.2208,  0.0763,  0.0705],
        [ 0.0775, -0.3007,  0.0842,  0.0358],
        [ 0.0212, -0.3065,  0.0431,  0.0697],
        [ 0.0372, -0.2482,  0.0791,  0.0302]], grad_fn=<AddmmBackward0>)

In [8]:

# for key, model in models.items():

# key = 'cnn'
key = 'seisLM'
model = models[key]


logits = models[key](waveforms)
loss = torch.nn.functional.cross_entropy(logits, labels)
predicted_labels = torch.argmax(logits, 1)


In [11]:
loss



tensor(1.4852, grad_fn=<NllLossBackward0>)